In [3]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import functions as f
import os

In [4]:
builder = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

spark = configure_spark_with_delta_pip(builder).getOrCreate()




In [5]:
sc = spark.sparkContext

Spark DataFrameWriter: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameWriter.mode.html#pyspark.sql.DataFrameWriter.mode

Spark to Delta mapping: https://docs.delta.io/latest/releases.html
Dataframe Reader and Writer versions: https://docs.delta.io/latest/versioning.html

Get Started: https://delta.io/learn/getting-started/

Deletes, Updates, Merges: https://docs.delta.io/latest/delta-update.html#language-python

In [ ]:
path_delta = "D:/Data/delta_test"

# Create a table and overwrite

In [6]:
sdf = spark.range(start=1, end=6, numPartitions=1)
sdf = sdf.withColumn("name", f.lit("Nikk"))
sdf.write.format("delta").mode("overwrite").save(path_delta)

In [7]:
spark.read.format("delta").load(path_delta).show()
deltaTable = DeltaTable.forPath(spark, path_delta)
deltaTable.detail().show(20, False)
deltaTable.history().show(20, False)

+---+----+
| id|name|
+---+----+
|  1|Nikk|
|  2|Nikk|
|  3|Nikk|
|  4|Nikk|
|  5|Nikk|
+---+----+

+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name|description|location                |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |0c92253f-b3e9-421b-aab3-83530419b76a|NULL|NULL       |file:/D:/Data/delta_test|2024-05-07 21:03:51.495|2024-05-07 21:03:57.359|[]              |1       |773    

In [8]:
files_parquet = [f"{path_delta}/_delta_log/{f}" for f in os.listdir(f"{path_delta}") if f.split(".")[-1] == "parquet"]
files_parquet

['D:/Data/delta_test/_delta_log/part-00000-7de78df9-2f39-46e9-a345-1364f2eaef27-c000.snappy.parquet']

# Append data

In [9]:
sdf = spark.range(start=6, end=11, numPartitions=1)
sdf = sdf.withColumn("name", f.lit("Marko"))
sdf.write.format("delta").mode("append").save(path_delta)

In [10]:
spark.read.format("delta").load(path_delta).show()
deltaTable = DeltaTable.forPath(spark, path_delta)
deltaTable.detail().show(20, False)
deltaTable.history().show(20, False)

+---+-----+
| id| name|
+---+-----+
|  6|Marko|
|  7|Marko|
|  8|Marko|
|  9|Marko|
| 10|Marko|
|  1| Nikk|
|  2| Nikk|
|  3| Nikk|
|  4| Nikk|
|  5| Nikk|
+---+-----+

+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name|description|location                |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |0c92253f-b3e9-421b-aab3-83530419b76a|NULL|NULL       |file:/D:/Data/delta_test|2024-05-07 2

In [11]:
files_parquet = [f"{path_delta}/_delta_log/{f}" for f in os.listdir(f"{path_delta}") if f.split(".")[-1] == "parquet"]
files_parquet

['D:/Data/delta_test/_delta_log/part-00000-4917efc4-6f13-4108-b0f3-021e8116f1cf-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-7de78df9-2f39-46e9-a345-1364f2eaef27-c000.snappy.parquet']

# Delete Data

In [12]:
deltaTable = DeltaTable.forPath(spark, path_delta)

# Using SQL formatted string
deltaTable.delete("id == 1")

# Using Spark SQL functions
#deltaTable.delete(f.col("id") == 1)

# Using SQL
#spark.sql(f"DELETE FROM delta.`{path_delta}` WHERE id == 1")


In [13]:
spark.read.format("delta").load(path_delta).show()
deltaTable = DeltaTable.forPath(spark, path_delta)
deltaTable.detail().show(20, False)
deltaTable.history().show(20, False)

+---+-----+
| id| name|
+---+-----+
|  6|Marko|
|  7|Marko|
|  8|Marko|
|  9|Marko|
| 10|Marko|
|  2| Nikk|
|  3| Nikk|
|  4| Nikk|
|  5| Nikk|
+---+-----+

+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name|description|location                |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |0c92253f-b3e9-421b-aab3-83530419b76a|NULL|NULL       |file:/D:/Data/delta_test|2024-05-07 21:03:51.495|

In [14]:
files_parquet = [f"{path_delta}/_delta_log/{f}" for f in os.listdir(f"{path_delta}") if f.split(".")[-1] == "parquet"]
files_parquet

['D:/Data/delta_test/_delta_log/part-00000-4917efc4-6f13-4108-b0f3-021e8116f1cf-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-7de78df9-2f39-46e9-a345-1364f2eaef27-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-dd9d11d7-d441-4f7e-8483-429591e0d13a-c000.snappy.parquet']

# Update data

In [15]:
deltaTable = DeltaTable.forPath(spark, path_delta)

# Using SQL formatted string
deltaTable.update(condition = "id = 7", set = { "name": "'Strahinja'"})

# Using Spark SQL functions
#deltaTable.update(condition = f.col("id") == 7, set = { "name": f.lit("Strahinja")})

# Using SQL
#spark.sql(f"UPDATE delta.`{path_delta}` SET name = 'Strahinja' WHERE id == 7")

In [16]:
spark.read.format("delta").load(path_delta).show()
deltaTable = DeltaTable.forPath(spark, path_delta)
deltaTable.detail().show(20, False)
deltaTable.history().show(20, False)

+---+---------+
| id|     name|
+---+---------+
|  6|    Marko|
|  7|Strahinja|
|  8|    Marko|
|  9|    Marko|
| 10|    Marko|
|  2|     Nikk|
|  3|     Nikk|
|  4|     Nikk|
|  5|     Nikk|
+---+---------+

+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name|description|location                |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+----+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |0c92253f-b3e9-421b-aab3-83530419b76a|NULL|NULL     

In [17]:
files_parquet = [f"{path_delta}/_delta_log/{f}" for f in os.listdir(f"{path_delta}") if f.split(".")[-1] == "parquet"]
files_parquet

['D:/Data/delta_test/_delta_log/part-00000-08ccc96a-c37b-44e3-821e-a9cab09798bd-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-4917efc4-6f13-4108-b0f3-021e8116f1cf-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-7de78df9-2f39-46e9-a345-1364f2eaef27-c000.snappy.parquet',
 'D:/Data/delta_test/_delta_log/part-00000-dd9d11d7-d441-4f7e-8483-429591e0d13a-c000.snappy.parquet']